# Модуль А. Загрузка и предобработка данных

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder

## 1.1 Загрузка данных

In [2]:
file = pd.read_csv('/'.join(os.getcwd().split('/')[:-1]) + '/diabetes_data/data.csv')
file

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


Техническими данными, судя по названию столбцов, являются следующие столбцы:
- encounter_id
- patient_nbr
- payer_code
- medical_specialty

Эти данные не следует держать в датасете, поэтому удалим их

In [3]:
file = file.drop(columns=['encounter_id', 'patient_nbr', 'payer_code', 'medical_specialty'])

## 1.2 Предобработка данных

Посчитаем количество `NaN` значений в каждом столбце. Если их количество больше определенного порога, то такие данные восстанавливать не имеет смысла. Значение `NaN` тут имеет либо символ `?`, либо пустая ячейка.

In [4]:
perc_dict = {'column': [], 'percentage': []}
for i in file.columns:
    perc_dict['column'].append(i)
    perc_dict['percentage'].append((file[i].isna().sum() + file[i].value_counts()['?'] if '?' in file[i].value_counts() else 0)/(101766) * 100)

show_perc = pd.DataFrame(perc_dict)
show_perc

,column,percentage
0,race,2.233555
1,gender,0.000000
2,age,0.000000
3,weight,96.858479
4,admission_type_id,0.000000
5,discharge_disposition_id,0.000000
6,admission_source_id,0.000000
7,time_in_hospital,0.000000
8,num_lab_procedures,0.000000
9,num_procedures,0.000000


Как видно, в этом датасете заполнены почти все поля, за исключением поля `weight`, в котором отсутствует больше 95% значений. Восстанавливать значения этого поля не имеет смысла, потому что на четырех процентах оставшихся данных выявить какие-либо закономерности если и возможно, то очень ресурсозатратно, поэтому удалим это поле.

In [5]:
file = file.drop(columns='weight')

Отлично. Заметим, что в поле `diag_1`, обозначающем основной диагноз, отсутствует 0,02% значений, или около 20 значений. Удалим строки, не содержащие значений в этом поле, поскольку диагноз станет важной частью нашего исследования.

In [6]:
rows_to_remove = []
for i in range(len(file)):
    if file['diag_1'][i] == '?':
        rows_to_remove.append(i)
file = file.drop(rows_to_remove)
file

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),1,3,7,3,51,0,16,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),1,4,5,5,33,3,18,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),1,1,7,1,53,0,9,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),2,3,7,10,45,2,21,...,No,Up,No,No,No,No,No,Ch,Yes,NO


Для дальнейшей работы нам потребуется заменить текстовые данные в столбцах на числовые. Для этого воспользуемся `LabelEncoder` из модуля `sklearn.preprocessing`. Ячейки со значением NaN (А это зачастую вопросительные знаки) будут также учтены и заменены своим кодом

Очевидно, что замена не потребуется в полях с типом `int64`

In [7]:
for i in file.columns:
    if file[i].dtype != 'int64':
        le = LabelEncoder()
        file[i] = le.fit_transform(file[i])
        if i == 'readmitted':
            print(dict(zip(range(len(le.classes_)),le.classes_)))
file

{0: '<30', 1: '>30', 2: 'NO'}


,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,3,0,0,6,25,1,1,41,0,1,...,0,1,1,0,0,0,0,1,0,2
1,3,0,1,1,1,7,3,59,0,18,...,0,3,1,0,0,0,0,0,1,1
2,1,0,2,1,1,7,2,11,5,13,...,0,1,1,0,0,0,0,1,1,2
3,3,1,3,1,1,7,2,44,1,16,...,0,3,1,0,0,0,0,0,1,2
4,3,1,4,1,1,7,1,51,0,8,...,0,2,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,1,1,7,1,3,7,3,51,0,16,...,0,0,1,0,0,0,0,0,1,1
101762,1,0,8,1,4,5,5,33,3,18,...,0,2,1,0,0,0,0,1,1,2
101763,3,1,7,1,1,7,1,53,0,9,...,0,0,1,0,0,0,0,0,1,2
101764,3,0,8,2,3,7,10,45,2,21,...,0,3,1,0,0,0,0,0,1,2


Сохраним полученный датасет в новый `csv` файл `preprocessed.csv`

In [8]:
file.to_csv('preprocessed.csv')